In [1]:
import pandas as pd
import altair as alt

# Load data

In [2]:
all_teams_2019_2024_df = pd.read_csv("all_teams_playoff_vs_season_stats_2019_2024.csv")
all_players_2019_2024_df = pd.read_csv("all_players_playoff_vs_season_stats_2019_2024.csv")

# Setup important selection tools

In [3]:
hover = alt.selection_point(on = 'pointerover',nearest = False, empty = 'none', clear = 'mouseout')
# click = alt.selection_point(empty = False, on = "click", name = "click")
team_select = alt.selection_point(fields=['Tm'], on='click', empty='none', name='TeamClick')

hover_and_click_condition_opacity = (alt.when(team_select).then(alt.value(1)).when(hover).then(alt.value(.75)).otherwise(alt.value(.50)))
hover_and_click_condition_size = (alt.when(team_select).then(alt.value(600)).when(hover).then(alt.value(200)).otherwise(alt.value(100)))

year_slider = alt.binding_range(
    min=2019, 
    max=2024, 
    step=1, 
    name='Select Year:'
)
year_selection = alt.selection_point(
    fields=['season_year'],
    bind=year_slider,
    value=2024  # Default selected year
)

In [4]:
def create_standings_chart():
    """
    Create a chart showing the standings of teams in the playoffs.

    Implicitly uses previously defined global variables and selections.
    """
    standings = alt.Chart(all_teams_2019_2024_df).mark_image(
        width=40,
        height=40
    ).encode(
        y=alt.Y('Rk_playoff:Q', scale=alt.Scale(domain=[16,1]), title='', axis=alt.Axis(grid=False)),
        x=alt.X('x:Q', axis=alt.Axis(grid=False, domain=False, domainOpacity=0), title=''),
        url='logo_url:N',
        tooltip=['Tm', 'W_playoff', 'L_playoff', 'season_year'],
        opacity=hover_and_click_condition_opacity,
    ).add_params(
        hover, team_select, year_selection  # Add the year selection parameter
    ).transform_filter(
        year_selection  # Filter data based on the selected year
    ).transform_calculate(
        x='0'
    ).properties(
        title='Playoff Rankings',
        height=725,
        width=100
    )
    return standings

def create_team_rating_chart(rating):
    chart = alt.Chart(all_teams_2019_2024_df).mark_image(width=30, height=30).encode(
        alt.X(f'{rating}_in_season:Q', title="Regular Season", scale=alt.Scale(zero=False, padding=10)),
        alt.Y(f'{rating}_playoff:Q', title="Playoffs", scale=alt.Scale(zero=False, padding=10)),
        url='logo_url:N',
        tooltip=[alt.Tooltip('Tm:N', title="Team"),
                alt.Tooltip('Rk_playoff', title='Rank in Playoff'),
                alt.Tooltip(f'{rating}_in_season', title=f"In Season {rating}"),
                alt.Tooltip(f'{rating}_playoff', title= "Playoff stat")],
        size=hover_and_click_condition_size,
        opacity = hover_and_click_condition_opacity
        # opacity=alt.condition(team_select, alt.value(1), alt.value(0.25))
    ).add_params(
        hover, team_select, year_selection
    ).transform_filter(
        year_selection
    ).properties(
        title = f"Team Level {rating}",
        height = 300,
        width = 300
    ).interactive()

    return chart


In [5]:
player_stat_options = ['PER', 'OBPM', 'DBPM', 'BPM', 'VORP', 'WS', "TS%"]
# Stat dropdown parameter
player_stat_param = alt.param(
    name='stat',
    bind=alt.binding_select(options=player_stat_options, name='Select Player Statistic:'),
    value='PER'  
)
player_stat_layer_width = 480
player_stat_layer_height = 320


def make_player_stat_layer(stat_name):
    # Player points chart
    stat_chart = alt.Chart(all_players_2019_2024_df).mark_circle(size=100).encode(
        x=alt.X(f'{stat_name}_in_season:Q', title='Regular Season', scale=alt.Scale(zero=False, padding=10)),
        y=alt.Y(f'{stat_name}_playoff:Q', title='Playoffs', scale=alt.Scale(zero=False, padding=10)),
        opacity=alt.condition(team_select, alt.value(.75), alt.value(0.25)),
        size=alt.condition(team_select, alt.value(150), alt.value(0)),
        tooltip=[
            alt.Tooltip('Player'),
            alt.Tooltip('Tm_playoff', title='Playoff Team'),
            alt.Tooltip(f'{stat_name}_playoff:Q', title='Playoff Stat'),
            alt.Tooltip(f'{stat_name}_in_season:Q', title='Regular Season Stat'),
        ],
        color=alt.condition(
            team_select,
            alt.Color('Tm_playoff:N', scale=alt.Scale(scheme='tableau10'), legend=None),
            alt.value('lightgray')
        )
    ).transform_calculate(
        stat=f'"{stat_name}"'
    ).transform_filter(
        'datum.stat === stat'
    ).transform_filter(
        year_selection
    ).properties(
        width=player_stat_layer_width,
        height=player_stat_layer_height,
        title=f'Player Level Statistics',
    )

    # Label overlay for this stat (constant dummy data)
    text_df = pd.DataFrame([{'x': 0, 'y': 1, 'stat': stat_name, 'season_year': y} for y in range(2019, 2025)])

    text_chart = alt.Chart(text_df).mark_text(
        align='left',
        baseline='top',
        dx=10,
        dy=10,
        fontSize=25,
        fontWeight='bold',
        color='black'
    ).encode(
        x=alt.value(0),
        y=alt.value(0),
        text=alt.value(stat_name),
        opacity=alt.condition('datum.stat === stat', alt.value(0.6), alt.value(0))
    ).transform_filter(
        'datum.stat === stat'
    ).transform_filter(
        year_selection
    ).properties(
        width=player_stat_layer_width,
        height=player_stat_layer_height
    )

    return stat_chart + text_chart


def create_player_stat_chart():
    # Compose full chart with all stat layers (but only one visible at a time)
    player_stats_chart = alt.layer(
        *[make_player_stat_layer(stat) for stat in player_stat_options]
    ).add_params(
        player_stat_param, hover, team_select, year_selection
    ).interactive()
    return player_stats_chart

In [6]:
team_stat_options = ['Pace', 'FTr', '3PAr', 'TS%']
# Stat dropdown parameter
team_stat_param = alt.param(
    name='team_stat',
    bind=alt.binding_select(options=team_stat_options, name='Select Team Statistic:'),
    value='Pace'  
)
team_stat_layer_width = 480
team_stat_layer_height = 320


def make_team_stat_layer(stat_name):
    stat_chart = alt.Chart(all_teams_2019_2024_df).mark_image(width=30, height=30).encode(
        alt.X(f'{stat_name}_in_season:Q', title="Regular Season", scale=alt.Scale(zero=False, padding=10)),
        alt.Y(f'{stat_name}_playoff:Q', title="Playoffs", scale=alt.Scale(zero=False, padding=10)),
        url='logo_url:N',
        tooltip=[alt.Tooltip('Tm:N', title="Team"),
                alt.Tooltip('Rk_playoff', title='Rank in Playoff'),
                alt.Tooltip(f'{stat_name}_in_season', title=f"In Season {stat_name}"),
                alt.Tooltip(f'{stat_name}_playoff', title= "Playoff stat")],
        size=hover_and_click_condition_size,
        opacity=hover_and_click_condition_opacity
    ).transform_calculate(
        stat=f'"{stat_name}"'
    ).transform_filter(
        'datum.stat === team_stat'
    ).transform_filter(
        year_selection
    ).properties(
        width=team_stat_layer_width,
        height=team_stat_layer_height,
        title=f'Team Level Statistics',
    )

    # Label overlay for this stat (constant dummy data)
    text_df = pd.DataFrame([{'x': 0, 'y': 1, 'stat': stat_name, 'season_year': y} for y in range(2019, 2025)])

    text_chart = alt.Chart(text_df).mark_text(
        align='left',
        baseline='top',
        dx=10,
        dy=10,
        fontSize=25,
        fontWeight='bold',
        color='black'
    ).encode(
        x=alt.value(0),
        y=alt.value(0),
        text=alt.value(stat_name),
        opacity=alt.condition('datum.stat === team_stat', alt.value(0.6), alt.value(0))
    ).transform_filter(
        'datum.stat === team_stat'
    ).transform_filter(
        year_selection
    ).properties(
        width=team_stat_layer_width,
        height=team_stat_layer_height
    )

    return stat_chart + text_chart


def create_team_stat_chart():
    # Compose full chart with all stat layers (but only one visible at a time)
    team_stats_chart = alt.layer(
        *[make_team_stat_layer(stat) for stat in team_stat_options]
    ).add_params(
        team_stat_param, hover, team_select, year_selection
    ).interactive()
    return team_stats_chart


In [7]:
create_team_stat_chart()

alt.LayerChart(...)

In [8]:
ratings = (
    ((create_team_rating_chart("NRtg") | create_team_rating_chart("ORtg") | create_team_rating_chart("DRtg"))
    &  ( create_team_stat_chart() | create_player_stat_chart() ))
)
(create_standings_chart() | ratings).properties(
    title=alt.Title(
        text='How do NBA teams perform in the playoffs relative to the regular season?',
        fontSize=25,
        subtitle=['Does any large change in these metrics in different season parts correlate with the Champion in a given year?'],
        subtitleFontSize=18,
        subtitleColor='gray',
    )
).save('playoff_vs_season_stats_2019_2024.html', format='html')